<a href="https://colab.research.google.com/github/DariusTheGeek/Radiant-Earth-Spot-the-Crop-Challenge/blob/main/Brainiac_Field_Aggregation_Mean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Use this notebook to calculate mean of pixels per field for 
  - Sentinel one data
  - Sentinel 2 data with 25 periods/dates
  - Sentinel 2 data with month start month end dates

**All you have to change is the path to the files**

In [ ]:
!pip -qq install pyspark findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz

     |████████████████████████████████| 212.4 MB 48 kB/s 
     |████████████████████████████████| 198 kB 50.4 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import os
import glob
from tqdm.notebook import tqdm_notebook
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import Row, DataFrame, Column
from pyspark.sql.functions import lit

from functools import reduce

In [ ]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
spark = SparkSession.builder \
    .master('local[*]') \
    .config("spark.driver.memory", "15g") \
    .appName('my-cool-app') \
    .getOrCreate()

In [ ]:
spark

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/CompeData/Radiant/train_file.csv')
field_ids = train.field.unique()
len(field_ids)

87114

In [ ]:
train.shape

(87114, 106)

## SAR

In [ ]:
%%time
dfs = []
for i in tqdm_notebook(glob.glob('/content/drive/MyDrive/CompeData/Radiant/s1_all_data/s1_train*.csv')):
  dfs.append(spark.read.csv(path = i, sep =',', encoding = 'UTF-8', comment = None, header = True))

  0%|          | 0/10 [00:00<?, ?it/s]

CPU times: user 165 ms, sys: 19.9 ms, total: 184 ms
Wall time: 11.1 s


In [ ]:
dfs[0].show(5)

+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+---+------+
|04_0_VV|04_0_VH|04_1_VV|04_1_VH|04_2_VV|04_2_VH|04_3_VV|04_3_VH|04_4_VV|04_4_VH|04_5_VV|04_5_VH|05_0_VV|05_0_VH|05_1_VV|05_1_VH|05_2_VV|05_2_VH|05_3_VV|05_3_VH|05_4_VV|05_4_VH|06_0_VV|06_0_VH|06_1_VV|06_1_VH|06_2_VV|06_2_VH|06_3_VV|06_3_VH|06_4_VV|06_4_VH|07_0_VV|07_0_VH|07_1_VV|07_1_VH|07_2_VV|07_2_VH|07_3_VV|07_3_VH|07_4_VV|07

In [ ]:
%%time
cols = set()
for j in tqdm_notebook(dfs):
    for x in j.columns:
        cols.add(x)
cols = sorted(cols)

# Create a dictionary with all the dataframes
all_data = {}
for i, d in enumerate(dfs):
    new_name = 'new_df' + str(i)  # New name for the key, the dataframe is the value
    all_data[new_name] = d
    # Loop through all column names. Add the missing columns to the dataframe (with value 0)
    for x in cols:
        if x not in d.columns:
            all_data[new_name] = all_data[new_name].withColumn(x, F.lit(0))
    all_data[new_name] = all_data[new_name].select(cols)  # Use 'select' to get the columns sorted

  0%|          | 0/10 [00:00<?, ?it/s]

CPU times: user 418 ms, sys: 97.2 ms, total: 515 ms
Wall time: 2.52 s


In [ ]:
len(dfs)

10

In [ ]:
%%time
all_data_dfs = [all_data['new_df'+str(i)] for i in range(10)]
final_data = reduce(DataFrame.unionAll, all_data_dfs)
final_data.show(5)

+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+---+
|04_0_VH|04_0_VV|04_1_VH|04_1_VV|04_2_VH|04_2_VV|04_3_VH|04_3_VV|04_4_VH|04_4_VV|04_5_VH|04_5_VV|05_0_VH|05_0_VV|05_1_VH|05_1_VV|05_2_VH|05_2_VV|05_3_VH|05_3_VV|05_4_VH|05_4_VV|06_0_VH|06_0_VV|06_1_VH|06_1_VV|06_2_VH|06_2_VV|06_3_VH|06_3_VV|06_4_VH|06_4_VV|07_0_VH|07_0_VV|07_1_VH|07_1_VV|07_2_VH|07_2_VV|07_3_VH|07_3_VV|07_4_VH|07

In [ ]:
final_data = final_data.select(*(F.col(c).cast("float").alias(c) for c in final_data.columns))
final_data.printSchema()

In [ ]:
%%time
final_data_mean = final_data.groupBy('fields').mean().toPandas()
final_data_mean.head()

CPU times: user 22.7 s, sys: 2.51 s, total: 25.2 s
Wall time: 59min 1s


In [ ]:
final_data_mean.head()

,fields,avg(04_0_VH),avg(04_0_VV),avg(04_1_VH),avg(04_1_VV),avg(04_2_VH),avg(04_2_VV),avg(04_3_VH),avg(04_3_VV),avg(04_4_VH),avg(04_4_VV),avg(04_5_VH),avg(04_5_VV),avg(05_0_VH),avg(05_0_VV),avg(05_1_VH),avg(05_1_VV),avg(05_2_VH),avg(05_2_VV),avg(05_3_VH),avg(05_3_VV),avg(05_4_VH),avg(05_4_VV),avg(06_0_VH),avg(06_0_VV),avg(06_1_VH),avg(06_1_VV),avg(06_2_VH),avg(06_2_VV),avg(06_3_VH),avg(06_3_VV),avg(06_4_VH),avg(06_4_VV),avg(07_0_VH),avg(07_0_VV),avg(07_1_VH),avg(07_1_VV),avg(07_2_VH),avg(07_2_VV),avg(07_3_VH),...,avg(08_1_VH),avg(08_1_VV),avg(08_2_VH),avg(08_2_VV),avg(08_3_VH),avg(08_3_VV),avg(08_4_VH),avg(08_4_VV),avg(09_0_VH),avg(09_0_VV),avg(09_1_VH),avg(09_1_VV),avg(09_2_VH),avg(09_2_VV),avg(09_3_VH),avg(09_3_VV),avg(09_4_VH),avg(09_4_VV),avg(10_0_VH),avg(10_0_VV),avg(10_1_VH),avg(10_1_VV),avg(10_2_VH),avg(10_2_VV),avg(10_3_VH),avg(10_3_VV),avg(10_4_VH),avg(10_4_VV),avg(11_0_VH),avg(11_0_VV),avg(11_1_VH),avg(11_1_VV),avg(11_2_VH),avg(11_2_VV),avg(11_3_VH),avg(11_3_VV),avg(11_4_VH),avg(11_4_VV),avg(fields),avg(y)
0,87958.0,2.676827,19.940902,2.954588,16.914774,1.517263,17.317885,2.625816,15.063453,2.490202,19.443235,2.078072,13.496112,3.026439,24.278383,3.165474,20.325661,4.285537,29.982271,3.807465,24.830171,3.390358,28.850078,2.199689,17.498600,4.494868,53.866874,3.990980,29.867496,3.549611,41.758009,3.655988,21.973250,2.746812,22.492379,3.851633,20.743390,4.149611,31.140280,4.115708,...,6.016485,23.416174,5.159876,17.355521,5.144323,19.175117,3.609331,9.614308,4.097667,11.322240,4.806843,11.568896,5.043857,14.158942,5.234526,13.975117,5.299222,16.238258,5.269362,14.159876,4.789114,17.764852,5.262830,19.453810,3.973872,16.525972,4.184137,13.902022,2.562364,15.148056,3.333126,14.901400,2.629238,17.477138,3.062519,14.792224,2.813997,18.101711,87958.0,6.0
1,26338.0,5.603060,26.801530,2.649399,27.394098,4.034973,20.696393,NaN,NaN,NaN,NaN,NaN,NaN,3.618798,21.302077,4.794973,25.068852,5.045683,27.548634,NaN,NaN,NaN,NaN,3.813552,42.742732,3.390164,31.265792,NaN,NaN,NaN,NaN,NaN,NaN,3.405246,17.995410,4.087869,25.903825,3.808743,16.022295,NaN,...,4.260765,15.931585,NaN,NaN,NaN,NaN,NaN,NaN,4.268415,11.620328,4.164372,11.830383,5.489836,15.177049,NaN,NaN,NaN,NaN,5.150601,15.444809,3.140765,15.039781,NaN,NaN,NaN,NaN,NaN,NaN,2.434973,15.283279,2.280000,15.930710,3.110820,17.633443,NaN,NaN,NaN,NaN,26338.0,7.0
2,55436.0,1.374696,12.523684,1.226113,12.234818,1.432591,12.479352,NaN,NaN,NaN,NaN,NaN,NaN,4.423279,26.419231,5.006680,28.885628,4.181377,26.215992,NaN,NaN,NaN,NaN,4.687045,35.269636,4.108704,28.842713,NaN,NaN,NaN,NaN,NaN,NaN,4.374089,24.329960,8.350405,42.986437,9.071457,36.728340,NaN,...,8.344939,25.227530,NaN,NaN,NaN,NaN,NaN,NaN,9.095344,20.195142,14.563158,32.720040,17.946559,38.577733,NaN,NaN,NaN,NaN,12.281984,28.092105,7.105263,18.811741,NaN,NaN,NaN,NaN,NaN,NaN,3.455466,13.469231,2.971053,15.141296,2.600810,14.885628,NaN,NaN,NaN,NaN,55436.0,8.0
3,101335.0,1.514648,9.358670,1.258116,7.973872,1.628662,8.589865,1.377672,8.021378,1.794141,10.216944,7.686461,44.382423,1.743468,11.716548,1.385590,9.500396,1.771971,12.045131,4.846397,29.173397,1.908947,13.020586,1.387173,10.524149,3.228029,29.858274,3.025337,21.114014,3.159145,20.205067,3.876485,20.482185,3.369755,14.817102,4.915281,22.543151,5.932700,25.900238,5.742676,...,6.458432,21.057007,5.045922,17.707838,5.969121,18.872526,3.909739,13.253365,4.026128,10.783848,4.828979,15.520190,3.944576,9.995249,4.441805,14.840063,5.030087,13.472684,8.271576,21.724466,5.296912,16.022169,6.555819,22.524149,3.804434,13.372130,3.058591,11.595408,4.068884,23.426762,1.793349,9.307205,1.619161,9.403009,1.338876,8.879652,1.653998,10.286619,101335.0,7.0
4,13268.0,3.000000,19.500000,5.166667,12.833333,3.666667,20.666667,2.000000,10.833333,3.666667,18.666667,2.333333,14.333333,2.833333,20.666667,1.666667,13.500000,2.333333,25.166667,4.166667,17.500000,2.000000,18.166667,3.333333,20.500000,6.000000,58.000000,3.833333,33.333333,2.166667,35.833333,2.333333,21.166667,4.333333,29.000000,3.833333,28.833333

In [ ]:
sar_train = final_data_mean.iloc[:, 1:]
sar_train.columns = final_data.columns
sar_train.y = round(sar_train.y).astype(int)
sar_train = sar_train[sar_train.y != 0].reset_index(drop = True)
sar_train.head()

,04_0_VH,04_0_VV,04_1_VH,04_1_VV,04_2_VH,04_2_VV,04_3_VH,04_3_VV,04_4_VH,04_4_VV,04_5_VH,04_5_VV,05_0_VH,05_0_VV,05_1_VH,05_1_VV,05_2_VH,05_2_VV,05_3_VH,05_3_VV,05_4_VH,05_4_VV,06_0_VH,06_0_VV,06_1_VH,06_1_VV,06_2_VH,06_2_VV,06_3_VH,06_3_VV,06_4_VH,06_4_VV,07_0_VH,07_0_VV,07_1_VH,07_1_VV,07_2_VH,07_2_VV,07_3_VH,07_3_VV,...,08_1_VH,08_1_VV,08_2_VH,08_2_VV,08_3_VH,08_3_VV,08_4_VH,08_4_VV,09_0_VH,09_0_VV,09_1_VH,09_1_VV,09_2_VH,09_2_VV,09_3_VH,09_3_VV,09_4_VH,09_4_VV,10_0_VH,10_0_VV,10_1_VH,10_1_VV,10_2_VH,10_2_VV,10_3_VH,10_3_VV,10_4_VH,10_4_VV,11_0_VH,11_0_VV,11_1_VH,11_1_VV,11_2_VH,11_2_VV,11_3_VH,11_3_VV,11_4_VH,11_4_VV,fields,y
0,2.676827,19.940902,2.954588,16.914774,1.517263,17.317885,2.625816,15.063453,2.490202,19.443235,2.078072,13.496112,3.026439,24.278383,3.165474,20.325661,4.285537,29.982271,3.807465,24.830171,3.390358,28.850078,2.199689,17.498600,4.494868,53.866874,3.990980,29.867496,3.549611,41.758009,3.655988,21.973250,2.746812,22.492379,3.851633,20.743390,4.149611,31.140280,4.115708,15.497356,...,6.016485,23.416174,5.159876,17.355521,5.144323,19.175117,3.609331,9.614308,4.097667,11.322240,4.806843,11.568896,5.043857,14.158942,5.234526,13.975117,5.299222,16.238258,5.269362,14.159876,4.789114,17.764852,5.262830,19.453810,3.973872,16.525972,4.184137,13.902022,2.562364,15.148056,3.333126,14.901400,2.629238,17.477138,3.062519,14.792224,2.813997,18.101711,87958.0,6
1,5.603060,26.801530,2.649399,27.394098,4.034973,20.696393,NaN,NaN,NaN,NaN,NaN,NaN,3.618798,21.302077,4.794973,25.068852,5.045683,27.548634,NaN,NaN,NaN,NaN,3.813552,42.742732,3.390164,31.265792,NaN,NaN,NaN,NaN,NaN,NaN,3.405246,17.995410,4.087869,25.903825,3.808743,16.022295,NaN,NaN,...,4.260765,15.931585,NaN,NaN,NaN,NaN,NaN,NaN,4.268415,11.620328,4.164372,11.830383,5.489836,15.177049,NaN,NaN,NaN,NaN,5.150601,15.444809,3.140765,15.039781,NaN,NaN,NaN,NaN,NaN,NaN,2.434973,15.283279,2.280000,15.930710,3.110820,17.633443,NaN,NaN,NaN,NaN,26338.0,7
2,1.374696,12.523684,1.226113,12.234818,1.432591,12.479352,NaN,NaN,NaN,NaN,NaN,NaN,4.423279,26.419231,5.006680,28.885628,4.181377,26.215992,NaN,NaN,NaN,NaN,4.687045,35.269636,4.108704,28.842713,NaN,NaN,NaN,NaN,NaN,NaN,4.374089,24.329960,8.350405,42.986437,9.071457,36.728340,NaN,NaN,...,8.344939,25.227530,NaN,NaN,NaN,NaN,NaN,NaN,9.095344,20.195142,14.563158,32.720040,17.946559,38.577733,NaN,NaN,NaN,NaN,12.281984,28.092105,7.105263,18.811741,NaN,NaN,NaN,NaN,NaN,NaN,3.455466,13.469231,2.971053,15.141296,2.600810,14.885628,NaN,NaN,NaN,NaN,55436.0,8
3,1.514648,9.358670,1.258116,7.973872,1.628662,8.589865,1.377672,8.021378,1.794141,10.216944,7.686461,44.382423,1.743468,11.716548,1.385590,9.500396,1.771971,12.045131,4.846397,29.173397,1.908947,13.020586,1.387173,10.524149,3.228029,29.858274,3.025337,21.114014,3.159145,20.205067,3.876485,20.482185,3.369755,14.817102,4.915281,22.543151,5.932700,25.900238,5.742676,20.196358,...,6.458432,21.057007,5.045922,17.707838,5.969121,18.872526,3.909739,13.253365,4.026128,10.783848,4.828979,15.520190,3.944576,9.995249,4.441805,14.840063,5.030087,13.472684,8.271576,21.724466,5.296912,16.022169,6.555819,22.524149,3.804434,13.372130,3.058591,11.595408,4.068884,23.426762,1.793349,9.307205,1.619161,9.403009,1.338876,8.879652,1.653998,10.286619,101335.0,7
4,3.000000,19.500000,5.166667,12.833333,3.666667,20.666667,2.000000,10.833333,3.666667,18.666667,2.333333,14.333333,2.833333,20.666667,1.666667,13.500000,2.333333,25.166667,4.166667,17.500000,2.000000,18.166667,3.333333,20.500000,6.000000,58.000000,3.833333,33.333333,2.166667,35.833333,2.333333,21.166667,4.333333,29.000000,3.833333,28.833333,5.833333,17.833333,4.833333,20.333333,...,3.333333,32.500000,7.500000,22.333333,4.500000,19.666667,4.333333,9.166667,2.166667,13.166667,8.500000,16.666667,3.833333,22.166667,2.833333,21.000000,1.333333,28.833333,2.500000,27.166667,2.500000,27.166667,3.333333,28.666667,2.166667,20.333333,2.166667,11.000000,2.333333,27.333333,3.000000,15.500000,5.333333,24.000000,9.166667,42.333333,5.000000,23.166667,13268.0,2


In [ ]:
sar_train.y.unique()

array([6, 7, 8, 2, 5, 4, 1, 3, 9])

In [ ]:
sar_train.to_csv('/content/drive/MyDrive/CompeData/Radiant/s1_all_data/sar_train.csv', index = False)

In [ ]:
%%time
dfs = []
for i in tqdm_notebook(glob.glob('/content/drive/MyDrive/CompeData/Radiant/s1_all_data/s1_test*.csv')):
  dfs.append(spark.read.csv(path = i, sep =',', encoding = 'UTF-8', comment = None, header = True))

  0%|          | 0/5 [00:00<?, ?it/s]

CPU times: user 89.9 ms, sys: 3.57 ms, total: 93.4 ms
Wall time: 1.99 s


In [ ]:
%%time
cols = set()
for j in tqdm_notebook(dfs):
    for x in j.columns:
        cols.add(x)
cols = sorted(cols)

# Create a dictionary with all the dataframes
all_data = {}
for i, d in enumerate(dfs):
    new_name = 'new_df' + str(i)  # New name for the key, the dataframe is the value
    all_data[new_name] = d
    # Loop through all column names. Add the missing columns to the dataframe (with value 0)
    for x in cols:
        if x not in d.columns:
            all_data[new_name] = all_data[new_name].withColumn(x, F.lit(0))
    all_data[new_name] = all_data[new_name].select(cols)  # Use 'select' to get the columns sorted

  0%|          | 0/5 [00:00<?, ?it/s]

CPU times: user 199 ms, sys: 28.2 ms, total: 227 ms
Wall time: 1.02 s


In [ ]:
len(dfs)

5

In [ ]:
%%time
all_data_dfs = [all_data['new_df'+str(i)] for i in range(5)]
final_data = reduce(DataFrame.unionAll, all_data_dfs)
final_data.show(5)

+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+
|04_0_VH|04_0_VV|04_1_VH|04_1_VV|04_2_VH|04_2_VV|04_3_VH|04_3_VV|04_4_VH|04_4_VV|04_5_VH|04_5_VV|05_0_VH|05_0_VV|05_1_VH|05_1_VV|05_2_VH|05_2_VV|05_3_VH|05_3_VV|05_4_VH|05_4_VV|06_0_VH|06_0_VV|06_1_VH|06_1_VV|06_2_VH|06_2_VV|06_3_VH|06_3_VV|06_4_VH|06_4_VV|07_0_VH|07_0_VV|07_1_VH|07_1_VV|07_2_VH|07_2_VV|07_3_VH|07_3_VV|07_4_VH|07_4_V

In [ ]:
final_data = final_data.select(*(F.col(c).cast("float").alias(c) for c in final_data.columns))
final_data.printSchema()

In [ ]:
%%time
final_data_mean = final_data.groupBy('fields').mean().toPandas()
final_data_mean.head()

CPU times: user 10.1 s, sys: 1.04 s, total: 11.1 s
Wall time: 25min 29s


In [ ]:
sar_test = final_data_mean.iloc[:, 1:]
sar_test.columns = final_data.columns
sar_test.head()

,04_0_VH,04_0_VV,04_1_VH,04_1_VV,04_2_VH,04_2_VV,04_3_VH,04_3_VV,04_4_VH,04_4_VV,04_5_VH,04_5_VV,05_0_VH,05_0_VV,05_1_VH,05_1_VV,05_2_VH,05_2_VV,05_3_VH,05_3_VV,05_4_VH,05_4_VV,06_0_VH,06_0_VV,06_1_VH,06_1_VV,06_2_VH,06_2_VV,06_3_VH,06_3_VV,06_4_VH,06_4_VV,07_0_VH,07_0_VV,07_1_VH,07_1_VV,07_2_VH,07_2_VV,07_3_VH,07_3_VV,...,08_0_VV,08_1_VH,08_1_VV,08_2_VH,08_2_VV,08_3_VH,08_3_VV,08_4_VH,08_4_VV,09_0_VH,09_0_VV,09_1_VH,09_1_VV,09_2_VH,09_2_VV,09_3_VH,09_3_VV,09_4_VH,09_4_VV,10_0_VH,10_0_VV,10_1_VH,10_1_VV,10_2_VH,10_2_VV,10_3_VH,10_3_VV,10_4_VH,10_4_VV,11_0_VH,11_0_VV,11_1_VH,11_1_VV,11_2_VH,11_2_VV,11_3_VH,11_3_VV,11_4_VH,11_4_VV,fields
0,0.816514,5.449541,0.651376,4.724771,0.917431,4.431193,NaN,NaN,NaN,NaN,NaN,NaN,0.715596,3.256881,0.761468,4.458716,0.853211,4.458716,NaN,NaN,NaN,NaN,0.825688,14.165138,0.954128,16.211009,NaN,NaN,NaN,NaN,NaN,NaN,1.284404,5.422018,0.880734,11.623853,0.853211,6.009174,NaN,NaN,...,23.899083,1.807339,16.972477,NaN,NaN,NaN,NaN,NaN,NaN,1.724771,9.348624,2.321101,7.706422,1.834862,8.174312,NaN,NaN,NaN,NaN,1.522936,6.715596,1.899083,10.045872,NaN,NaN,NaN,NaN,NaN,NaN,1.862385,7.807339,1.972477,8.275229,1.110092,7.440367,NaN,NaN,NaN,NaN,714.0
1,0.954545,22.136364,1.636364,11.272727,2.863636,9.181818,NaN,NaN,NaN,NaN,NaN,NaN,1.545455,9.318182,2.363636,8.272727,1.818182,10.772727,NaN,NaN,NaN,NaN,1.590909,15.363636,2.000000,8.545455,NaN,NaN,NaN,NaN,NaN,NaN,2.363636,12.863636,3.181818,11.136364,2.590909,6.818182,NaN,NaN,...,7.454545,1.545455,10.590909,NaN,NaN,NaN,NaN,NaN,NaN,1.227273,11.136364,3.227273,9.772727,1.772727,12.136364,NaN,NaN,NaN,NaN,2.818182,11.409091,2.181818,9.954545,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,9.136364,2.136364,9.318182,1.954545,9.590909,NaN,NaN,NaN,NaN,118859.0
2,1.137255,7.758170,0.705882,4.725490,1.000000,7.254902,0.882353,5.594771,1.346405,7.431373,2.836601,16.104575,1.104575,6.882353,1.000000,5.496732,1.418301,7.549020,0.856209,6.248366,1.287582,7.006536,0.856209,6.241830,1.993464,23.379085,2.111111,16.274510,2.895425,19.019608,2.647059,14.287582,4.130719,17.235294,3.980392,14.522876,3.823529,22.444444,4.241830,17.372549,...,14.751634,4.640523,20.758170,4.431373,21.209150,5.947712,19.209150,5.875817,21.725490,4.901961,15.790850,5.248366,16.254902,3.254902,10.856209,4.143791,11.830065,2.732026,7.869281,5.267974,12.862745,2.732026,7.803922,2.150327,10.111111,2.294118,8.830065,2.359477,6.869281,2.235294,15.287582,2.339869,7.542484,1.777778,8.679739,1.627451,7.026144,2.424837,9.052288,97477.0
3,2.223183,7.326990,1.904844,6.553633,1.930796,9.442907,1.871972,6.891003,2.129758,7.645329,3.726644,15.249135,1.762976,6.420415,1.610727,7.237024,1.946367,8.410035,2.178201,9.287197,2.411765,9.077855,1.896194,9.363322,2.603806,12.671280,2.429066,13.475779,2.859862,11.269896,2.169550,10.060554,2.768166,9.553633,2.131488,10.072664,2.695502,11.344291,2.266436,9.012111,...,9.608997,3.200692,17.339100,3.304498,12.577855,3.567474,13.972318,3.399654,11.550173,3.608997,10.935986,3.008651,9.434256,3.266436,10.474048,2.955017,10.346021,2.764706,11.133218,2.752595,9.804498,2.750865,10.935986,2.484429,11.025952,2.804498,10.377163,2.519031,9.332180,2.468858,13.237024,2.385813,9.250865,2.960208,9.309689,2.318339,8.731834,3.013841,10.859862,65754.0
4,2.090909,14.045455,2.462121,11.409091,3.969697,19.333333,2.590909,12.227273,2.742424,12.318182,3.454545,12.780303,2.325758,16.598485,3.757576,22.780303,2.954545,21.439394,2.848485,21.765152,3.090909,23.909091,3.530303,39.219697,7.393939,92.030303,6.325758,65.325758,4.878788,64.098485,7.045455,52.803030,3.257576,39.045455,5.962121,51.613636,4.257576,51.265152,4.886364,34.128788,...,29.333333,6.909091,72.530303,8.939394,51.833333,9.431818,64.310606,3.659091,20.946970,3.484848,22.159091,5.333333,20.787879,3.469697,16.810606,3.984848,12.227273,4.583333,15.893939,4.325758,10.204545,2.856061,9.325758,6.515152,10.166667,3.840909,8.363636,4.325758,7.265152,9.537879,25.234848,4.272727,10.431818,3.462121,16.500000,3.075758,13.515152,3.606061,18.939394,120372.0


In [ ]:
sar_test.to_csv('/content/drive/MyDrive/CompeData/Radiant/s1_all_data/sar_test.csv', index = False)
